# Data Clean Up 

In [33]:
# Import modules
import os
import pandas as pd

## Make library data set

In [34]:
# Set parameters
covid_path = "../Resources/Weekly_COVID-19_Cases__Tests__and_Deaths_by_ZIP_Code_pulled_272022.csv"
library_circulation_2019_path = "../Resources/Libraries_-_2019_Circulation_by_Location_pulled_272022.csv"
library_visitors_2019_path = "../Resources/Libraries_-_2019_Visitors_by_Location_pulled_272022.csv"
library_circulation_2020_path = "../Resources/Libraries_-_2020_Circulation_by_Location_pulled_272022.csv"
library_visitors_2020_path = "../Resources/Libraries_-_2020_Visitors_by_Location_pulled_272022.csv"
library_circulation_2021_path = "../Resources/Libraries_-_2021_Circulation_by_Location_pulled_272022.csv"
library_visitors_2021_path = "../Resources/Libraries_-_2021_Visitors_by_Location_pulled_272022.csv"

In [35]:
# Load datasets
covid_df = pd.read_csv(covid_path)
library_circulation_2019_df = pd.read_csv(library_circulation_2019_path, dtype={'ZIP': object})
library_visitors_2019_df = pd.read_csv(library_visitors_2019_path, dtype={'ZIP': object})
library_circulation_2020_df = pd.read_csv(library_circulation_2020_path, dtype={'ZIP': object})
library_visitors_2020_df = pd.read_csv(library_visitors_2020_path, dtype={'ZIP': object})
library_circulation_2021_df = pd.read_csv(library_circulation_2021_path, dtype={'ZIP': object})
library_visitors_2021_df = pd.read_csv(library_visitors_2021_path, dtype={'ZIP': object})

# covid_df.head()
library_circulation_2019_df.head()
# library_visitors_2019_df.head()
# library_circulation_2020_df.head()
# library_visitors_2020_df.head()
# library_circulation_2021_df.head()
# library_visitors_2021_df.head()
# library_visitors_2021_df

,LOCATION,ADDRESS,CITY,ZIP,JANUARY,FEBRUARY,MARCH,APRIL,MAY,JUNE,JULY,AUGUST,SEPTEMBER,OCTOBER,NOVEMBER,DECEMBER,YTD
0,Albany Park,3401 W. Foster Ave.,Chicago,60625,8214,7614,8460,7414,7044,7970,9664,9069,7053,8012,6571,5924,93009
1,Altgeld,13281 S. Corliss Ave.,Chicago,60827,378,326,332,270,320,325,354,229,234,377,369,500,4014
2,Archer Heights,5055 S. Archer Ave.,Chicago,60632,5365,5019,5417,5349,4833,4627,5356,4793,4537,5263,4051,3415,58025
3,Austin,5615 W. Race Ave.,Chicago,60644,215,590,706,736,806,983,1134,1236,1193,1361,1286,1082,11328
4,Austin-Irving,6100 W. Irving Park Rd.,Chicago,60634,9678,9764,10845,10541,9763,11442,11634,11063,9379,9932,8567,6492,119100


In [36]:
# For library circulation we have data without zip code (online renewal, media download, etc.)
# We want to drop this data since it has no zip code associated with it
library_circulation_2019_df = library_circulation_2019_df.dropna(how='any')
library_circulation_2020_df = library_circulation_2020_df.dropna(how='any')
library_circulation_2021_df = library_circulation_2021_df.dropna(how='any')

In [37]:
# Align zip code column name
covid_df = covid_df.rename(columns={"ZIP CODE": "ZIP"})
covid_df.head()

,ZIP Code,Week Number,Week Start,Week End,Cases - Weekly,Cases - Cumulative,Case Rate - Weekly,Case Rate - Cumulative,Tests - Weekly,Tests - Cumulative,...,Test Rate - Cumulative,Percent Tested Positive - Weekly,Percent Tested Positive - Cumulative,Deaths - Weekly,Deaths - Cumulative,Death Rate - Weekly,Death Rate - Cumulative,Population,Row ID,ZIP Code Location
0,60612,15,4/5/2020,4/11/2020,69.0,202.0,201.0,588.7,284.0,803,...,2340.4,0.3,0.3,3,7,8.7,20.4,34311,60612-2020-15,POINT (-87.687011 41.88004)
1,60612,16,4/12/2020,4/18/2020,66.0,268.0,192.0,781.1,286.0,1089,...,3173.9,0.3,0.3,3,10,8.7,29.1,34311,60612-2020-16,POINT (-87.687011 41.88004)
2,60604,4,1/23/2022,1/29/2022,8.0,322.0,1023.0,41176.5,246.0,11461,...,1465601.0,0.0,0.0,0,0,0.0,0.0,782,60604-2022-4,POINT (-87.629029 41.878153)
3,60615,24,6/7/2020,6/13/2020,12.0,377.0,29.0,907.1,538.0,3986,...,9590.3,0.0,0.1,0,15,0.0,36.1,41563,60615-2020-24,POINT (-87.602725 41.801993)
4,60615,25,6/14/2020,6/20/2020,13.0,390.0,31.0,938.3,551.0,4537,...,10916.0,0.0,0.1,1,16,2.4,38.5,41563,60615-2020-25,POINT (-87.602725 41.801993)


In [38]:
# Change LOCATION to BRANCH for 2019 data to match 2020 and 2021 format
library_circulation_2019_df = library_circulation_2019_df.rename(columns={"LOCATION": "BRANCH"})
library_visitors_2019_df = library_circulation_2019_df.rename(columns={"LOCATION": "BRANCH"})

# Change Location to LOCATION
library_circulation_2021_df = library_circulation_2021_df.rename(columns={"Location": "LOCATION"})
library_circulation_2021_df.head()

,BRANCH,ADDRESS,CITY,ZIP,JANUARY,FEBRUARY,MARCH,APRIL,MAY,JUNE,JULY,AUGUST,SEPTEMBER,OCTOBER,NOVEMBER,DECEMBER,YTD,LOCATION
0,Walker,11071 S. Hoyne Ave.,Chicago,60643,800,705,850,803,815,953,1098,914,880,1060,891,772,10541,POINT (-87.67426 41.692473)
1,Albany Park,3401 W. Foster Ave.,Chicago,60625,4274,3486,5170,4864,4561,5711,5983,5675,5603,5699,5370,5238,61634,POINT (-87.713595 41.975693)
2,Avalon,8148 S. Stony Island Ave.,Chicago,60617,1342,1099,1241,1264,1368,1378,1242,1432,1589,1476,1355,1217,16003,POINT (-87.585869 41.746322)
3,Brainerd,1350 W. 89th St.,Chicago,60620,311,349,501,60,85,278,197,280,277,337,314,332,3321,POINT (-87.657726 41.732282)
4,Oriole Park,7454 W. Balmoral Ave.,Chicago,60656,4375,3797,4725,4339,4386,5128,5511,4719,4630,5156,4850,3057,54673,POINT (-87.813329 41.978015)


In [39]:
# Add type column
library_circulation_2019_df["TYPE"] = ["Circulation"] * len(library_circulation_2019_df)
library_visitors_2019_df["TYPE"] = ["Visitors"] * len(library_visitors_2019_df)
library_circulation_2020_df["TYPE"] = ["Circulation"] * len(library_circulation_2020_df)
library_visitors_2020_df["TYPE"] = ["Visitors"] * len(library_visitors_2020_df)
library_circulation_2021_df["TYPE"] = ["Circulation"] * len(library_circulation_2021_df)
library_visitors_2021_df["TYPE"] = ["Visitors"] * len(library_visitors_2021_df)

# Add year column
library_circulation_2019_df["YEAR"] = [2019] * len(library_circulation_2019_df)
library_visitors_2019_df["YEAR"] = [2019] * len(library_visitors_2019_df)
library_circulation_2020_df["YEAR"] = [2020] * len(library_circulation_2020_df)
library_visitors_2020_df["YEAR"] = [2020] * len(library_visitors_2020_df)
library_circulation_2021_df["YEAR"] = [2021] * len(library_circulation_2021_df)
library_visitors_2021_df["YEAR"] = [2021] * len(library_visitors_2021_df)

In [40]:
# Merge circulation and visitors together
merged_2019 = pd.merge(library_circulation_2019_df, library_visitors_2019_df, how = "outer")
merged_2020 = pd.merge(library_circulation_2020_df, library_visitors_2020_df, how = "outer")
merged_2021 = pd.merge(library_circulation_2021_df, library_visitors_2021_df, how = "outer")

# Merge all together
merged = pd.merge(merged_2019, merged_2020, how = "outer")
merged = pd.merge(merged, merged_2021, how = "outer")
# merged.head()
merged.tail()

,BRANCH,ADDRESS,CITY,ZIP,JANUARY,FEBRUARY,MARCH,APRIL,MAY,JUNE,JULY,AUGUST,SEPTEMBER,OCTOBER,NOVEMBER,DECEMBER,YTD,TYPE,YEAR,LOCATION
480,Woodson Regional Library,9525 S. Halsted St.,Chicago,60628,3655,3446,5049,6815,3638,3144,5060,5636,5256,5312,4550,4426,55987,Visitors,2021,POINT (-87.643122 41.720675)
481,Thurgood Marshall,7506 S. Racine Ave.,Chicago,60620,2094,2043,2695,2698,2419,2557,2944,3165,2994,3092,2690,2773,32164,Visitors,2021,POINT (-87.65393 41.757373)
482,Chicago Bee,3647 S. State St.,Chicago,60609,496,488,595,653,866,974,898,1130,1076,1145,931,936,10188,Visitors,2021,POINT (-87.626436 41.828152)
483,Vodak-East Side,3710 E. 106th St.,Chicago,60617,2007,2769,2370,2357,2184,2188,3058,3058,3141,3674,3405,3357,33568,Visitors,2021,POINT (-87.533513 41.70272)
484,Sherman Park,5440 S. Racine Ave.,Chicago,60609,1420,1264,1699,1644,1404,1707,1809,1824,1669,1587,1489,1321,18837,Visitors,2021,POINT (-87.65487 41.794843)


In [8]:
# Export data
merged.to_csv("../Data/merged_library.csv")

In [18]:
covid_df.head()
# len(covid_df)

covid_df['Year'] = pd.to_datetime(covid_df['Week Start']).dt.year
covid_df['Month'] = pd.to_datetime(covid_df['Week Start']).dt.month
covid_df.head()
# covid_df['Year'].unique()

,ZIP,Week Number,Week Start,Week End,Cases - Weekly,Cases - Cumulative,Case Rate - Weekly,Case Rate - Cumulative,Tests - Weekly,Tests - Cumulative,...,Percent Tested Positive - Cumulative,Deaths - Weekly,Deaths - Cumulative,Death Rate - Weekly,Death Rate - Cumulative,Population,Row ID,ZIP Code Location,Month,Year
0,60612,15,4/5/2020,4/11/2020,69.0,202.0,201.0,588.7,284.0,803,...,0.3,3,7,8.7,20.4,34311,60612-2020-15,POINT (-87.687011 41.88004),4,2020
1,60612,16,4/12/2020,4/18/2020,66.0,268.0,192.0,781.1,286.0,1089,...,0.3,3,10,8.7,29.1,34311,60612-2020-16,POINT (-87.687011 41.88004),4,2020
2,60604,4,1/23/2022,1/29/2022,8.0,322.0,1023.0,41176.5,246.0,11461,...,0.0,0,0,0.0,0.0,782,60604-2022-4,POINT (-87.629029 41.878153),1,2022
3,60615,24,6/7/2020,6/13/2020,12.0,377.0,29.0,907.1,538.0,3986,...,0.1,0,15,0.0,36.1,41563,60615-2020-24,POINT (-87.602725 41.801993),6,2020
4,60615,25,6/14/2020,6/20/2020,13.0,390.0,31.0,938.3,551.0,4537,...,0.1,1,16,2.4,38.5,41563,60615-2020-25,POINT (-87.602725 41.801993),6,2020
